# Implementation of the Eddy Viscosity (EV) wake model in PyWake

This notebook describes the implementation of the Eddy Viscosity (EV) wind turbine wake model in PyWake. It outlines the system of governing equations and discussed the computational approach to the solution along with integration in the PyWake framework.

## Introduction

The Eddy Viscosity (EV) wind turbine wake model developed by Ainslie (1988) is widely used in the wind energy industry for pre-construction energy yield assessments (EYAs), sometimes with various modifications and extensions. It is implemented in many industry-standard EYA modelling tools, such as WindFarmer and Openwind. This notebook describes the implementation of this model in PyWake.

The motivation for implementing the EV model in PyWake is to allow for more convenient execution of wake loss calculations with this model, replication of legacy wake computations that rely on this model and inclusion of this model in benchmarking exercises. Whilst the EV model is not the most interesting from the perspective of academic research, it is useful for benchmarking due to its wide adoption in industry.

Anderson (2011) developed a simplified solution to the EV model equations by observing that the wake profile is self-similar in the streamwise direction. His approach allows for a reduction of the system of governing equations from two dimensions (steam-wise and radial distance from the wind turbine rotor centre) to one dimension (streamwise distance only), making them considerably easier and faster to solve. Because this simplification sacrifices neither accuracy nor precision of the solution to any material extent, it appears the obvious choice of approach and has been adopted in the EV model implementation in PyWake.

## Formulation

### Simplified formulation for centreline wake velocity

Anderson (2011) provided the following simplified formulation (in somewhat different notation) of the governing equations of fluid motion in the wake of a single wind turbine:
$$
\frac{\textrm{d}\tilde{u}_{c}}{\textrm{d}\tilde{x}} = \frac{16 \, \tilde{\epsilon} \left( \tilde{u}_{c}^{3} - \tilde{u}_{c}^{2} - \tilde{u}_{c} + 1 \right)}{C_{t} \, \tilde{u}_{c}},
$$
where $\tilde{u}_{c}$ is the dimensionless streamwise component of the wind centreline velocity in the wake, $\tilde{x}$ the non-dimensional downwind distance from the rotor, $C_{t}$ the wind turbine thrust coefficient and $\tilde{\epsilon}$ the non-dimensional wake eddy viscosity.

The non-dimensional quantities $\tilde{u}_{c}$ and $\tilde{x}$ are defined as
$$
  \tilde{u}_{c} = \frac{u_{c}}{U_{0}} \textrm{, and}
$$
$$
  \tilde{x} = \frac{x}{D},
$$
where $u_{c}$ is the dimensional streamwise wake centreline wind speed, $U_{0}$ the free-stream (undisturbed incident) wind speed, $x$ the dimensional downtime distance from the rotor and $D$ the diameter of the wind turbine rotor.

The thrust coefficient is defined as
$$
C_{t} = \frac{F_{t}}{\frac{1}{2} \rho A \, U_{0}^{2}}
$$
where $F_{t}$ is the streamwise thrust force on the rotor, $\rho$ the air density and $A = \pi (\frac{D}{2})^{2}$ the rotor swept area. It is noted that $F_{t}$ varies with $U_{0}$ and depends on the wind turbine control. Typically $C_{t}$ specifications are provided by a wind turbine OEM as tabulated binned values as a function of $U_{0}$, together with the power curve specification.

The non-dimensional wake eddy viscosity is a function of $\tilde{u}_{c}$ and $\tilde{x}$, defined as
$$
\tilde{\epsilon} (\tilde{x}, \tilde{u}_{c}) =
  F(\tilde{x}) \left(
    0.015 \left( \sqrt{\frac{3.56 C_{t}}{4 \left( 1 - \tilde{u}_{c}^{2} \right)}} \right) \left( 1 - \tilde{u}_{c} \right)+ 0.16 I_{0}
  \right)
$$
where $I_{0}$ is the ambient (upstream) turbulence intensity and $F(x)$ a filter function to adjust for near-wake mixing. Note that Ainslie (1988) and Anderson (2011) define $I_{0}$ as a percentage value and the definition here uses the unscaled standard non-dimensional quantity. Hence there is a difference of a factor of 100.

Ainslie (1988) defined the filter function as follows (also used in the formulation by Anderson, 2011):
$$
F(\tilde{x}) = \left\{
    0.65 + \left( \frac{\tilde{x} - 4.5}{23.32} \right) ^{\frac{1}{3}} \quad \textrm{if } \tilde{x} < 5.5 \textrm{, and} \\
    1.0                                                                \quad \textrm{if } \tilde{x} \geq 5.5.
\right.
$$

The WindFarmer Theory Manual explains that $F$ is fixed at unity in the WindFarmer implementation of the EV model based on validation against measurements reported by Taylor (1990). In Openwind, the near wake mixing filter function can be enabled or disabled by the user, where the default setting is keeping it enabled.

The governing differential equation expresses the downwind rate of change of $\tilde{u}_{c}$ with downwind distance $\tilde{x}$ as a function of the values of $\tilde{u}_{c}$ and $\tilde{x}$. Given an initial value, it can be solved for downwind. Ainslie (1988) proposed adopting the following initial value, which was also used by Anderson (2011):
$$
\tilde{u}_{c} (\tilde{x} = 2) = 1 - C_{t} + 0.05 + (16 C_{t} - 0.5) \left( \frac{I_{0}}{10} \right).
$$

In the near wake, for values of $\tilde{x}$ less than 2, $\tilde{u}_{c}$ is not defined. The EV model is not applicable in the near wake as it does not account for pressure gradients, which play an important role in the flow near to the rotor.

### Simplified formulation for centreline wake deficit

The formulation of the system of equations for the wake centreline velocity in the [previous section](#simplified-formulation-for-centreline-wake-velocity) can be rearranged in terms of the velocity deficit. This is a somewhat more convenient form to work with and fits more directly in the PyWake framework, since a PyWake wake deficit model is expected to provide wind speed deficit values rather than wind speed values. The non-dimensional centreline wake velocity deficit, $\tilde{u}_{d_{c}}$, is governed by the following equation:

$$
\frac{\textrm{d}\tilde{u}_{d_{c}}}{\textrm{d}\tilde{x}} =
  \frac{-16 \, \tilde{\epsilon} \, \tilde{u}_{d_{c}}^{2} \left( 2 - \tilde{u}_{d_{c}} \right)}{C_{t} \left( 1 - \tilde{u}_{d_{c}} \right)}.
$$

The eddy viscosity function can be expressed as:
$$
\tilde{\epsilon} (\tilde{x}, \tilde{u}_{d_{c}}) =
  F(\tilde{x}) \left(
    0.015 \, \tilde{u}_{d_{c}} \left( \sqrt{\frac{3.56 C_{t}}{4 \, \tilde{u}_{d_{c}} \left( 2 - \tilde{u}_{d_{c}} \right)}} \right) + 0.16 I_{0}.
  \right)
$$

When expressed in terms of $\tilde{u}_{d_{c}}$, the equation for the initial value becomes
$$
\tilde{u}_{d_{c}} (\tilde{x} = 2) = C_{t} - 0.05 - (16 C_{t} - 0.5) \left( \frac{I_{0}}{10} \right).
$$

### Wake width

The wake deficit profile is assumed to take a Gaussian form (Ainslie 1988, Anderson 2011):
$$
\tilde{u}_{d} (\tilde{x}, \tilde{r}) = \tilde{u}_{d_{c}} (\tilde{x}) \, \exp \left( -3.56 \left( \frac{\tilde{r}}{\tilde{w}} \right)^{2} \right),
$$
where $\tilde{r} = \frac{r}{D}$ is the non-dimensional radial distance from the wake centreline and $\tilde{w} = \frac{w}{D}$ is the non-dimensional wake width, defined as an estimate of the full width of the wake.

From conservation of momentum, $\tilde{w}$ can be expressed as (Ainslie 1988, Anderson 2011):
$$
\tilde{w} = \sqrt{\frac{3.56 C_{t}}{4 \, \tilde{u}_{d_{c}} \left( 2 - \tilde{u}_{d_{c}} \right)}}.
$$

### Wake turbulence model

The EV wake deficit model is often implemented in conjunction with the wake added turbulence model proposed by Quarton and Ainslie (1990). Both the WindFarmer and Openwind implementations uses a version of this wake turbulence model to adjust the incident turbulence intensity input to the EV model to account for added turbulence from the wakes of upstream turbines (i.e. for turbines not in the free wind stream).

The original empirical formulation proposed by Quarton and Ainslie (1990) provides the following equation for the added turbulence:
$$
I_{+} = 4.8 \, C_{t}^{0.7} \, I_{0}^{0.68} \left( \frac{x}{x_{n}} \right)^{-0.57},
$$
where $I_{+}$ is the wake added turbulence intensity estimate, $x_{n}$ is an estimate of the near wake length and the other terms are as defined previously.

Hassan (1992) proposed the following modified empirical formulation based on new experimental data:
$$
I_{+} = 5.7 \, C_{t}^{0.7} \, I_{0}^{0.68} \left( \frac{x}{x_{n}} \right)^{-0.96}.
$$

The PyWake implementation includes both variations, with the modified formulation used as default. Both WindFarmer and Openwind use the modified formulation.

The estimate of the near wake length $x_{n}$ is based upon an empirical calculation method proposed by Vermeulen and colleagues (Vermeulen 1980; Vermeulen and Builtjes 1981; Vermeulen and Vijge 1981), which can be expressed as
$$
x_{n} = \frac{n \, r_{0}}{\frac{\textrm{d}r}{\textrm{d}x}}.
$$
The parameter $n$ is calculated as
$$
n = \frac{n_{1} \left( 1 - n_{2} \right)}{n_{2} \left( 1 - n_{1} \right)}, \enspace
n_{1} = \sqrt{0.214 + 0.144 \, m}, \enspace
n_{2} = \sqrt{0.134 + 0.124 \, m},
$$
where
$$
m = \frac{1}{\sqrt{1 - C_{t}}}.
$$
The parameter $r_{0}$ is calculated as
$$
r_{0} = \frac{D}{2} \sqrt{\frac{m + 1}{2}}.
$$
The wake growth rate $\frac{\textrm{d}r}{\textrm{d}x}$ is calculated as
$$
\frac{\textrm{d}r}{\textrm{d}x} = \sqrt{\left(\frac{\textrm{d}r}{\textrm{d}x}\right)_{\alpha}^2 + \left(\frac{\textrm{d}r}{\textrm{d}x}\right)_{m}^2 + \left(\frac{\textrm{d}r}{\textrm{d}x}\right)_{\lambda}^2},
$$
where the term $\left(\frac{\textrm{d}r}{\textrm{d}x}\right)_{\alpha}$ is the contribution from ambient turbulence and calculated as
$$
\left(\frac{\textrm{d}r}{\textrm{d}x}\right)_{\alpha} = 2.5 \, I_{0} + 0.005,
$$
the term $\left(\frac{\textrm{d}r}{\textrm{d}x}\right)_{m}$ is the contribution from turbulence generated by shear forces and calculated as
$$
\left(\frac{\textrm{d}r}{\textrm{d}x}\right)_{m} = \frac{\left(1 - m\right) \sqrt{1.49 + m}}{9.76 \left(1 + m\right)}
$$
the term $\left(\frac{\textrm{d}r}{\textrm{d}x}\right)_{\lambda}$ is the contribution from mechanical turbulence calculated as
$$
\left(\frac{\textrm{d}r}{\textrm{d}x}\right)_{\lambda} = 0.012 \, B \, \lambda,
$$
where $B$ is the number of rotor blades and $\lambda$ is the tip speed ratio.

The wake added turbulence intensity calculated by this model provides an estimate of additional turbulence intensity relative to the freestream wind speed. For values relative to the reduced wind speed in the wake, the results need to be re-normalised relative to the waked wind speed.

## Computational approach

Considering the wake centreline deficit formulation, the [governing equations](#simplified-formulation-for-centreline-wake-deficit) can be combined into a single differential equation where $\frac{\textrm{d}}{\textrm{d}\tilde{x}}(\tilde{u}_{d_{c}})$ is a function of $\tilde{u}_{d_{c}}$, $\tilde{x}$, $C_{t}$ and $I_{0}$. For given values of $C_{t}$ and $I_{0}$, the initial value problem can be solved over the range of relevant downwind distances ($\tilde{x}$), starting at the initial $\tilde{u}_{d_{c}}$ value defined two rotor diameters downstream. Thus a three dimensional array of $\tilde{u}_{d_{c}}$ can be pre-computed over relevant coordinates along the dimensions $\tilde{x}$, $C_{t}$ and $I_{0}$.

The same approach can of course also be taken for the original wake centreline wind speed formulation. The PyWake implementation includes both the wind speed formulation and the deficit formulation. The results (lookup tables) arrived at from using the two alternative formulations were found to be very close.

The use of a pre-computed lookup tables is more efficient than solving the initial value problem for each individual case in a model run, since the number of cases is large for realistic scenarios. A vectorised solver could readily be built with the [SciPy IVP solver](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html), using the explicit Runge-Kutta method "RK45".

For each value $\tilde{u}_{d_{c}}(\tilde{x}, C_{t}, I_{0})$, a corresponding value $\tilde{w}(\tilde{x}, C_{t}, I_{0})$ can be computed from equation \ref{eq:wake_width}. Hence a three-dimensional array of $\tilde{w}$ with the same coordinates as the array for $\tilde{u}_{d_{c}}$ can be derived.

From the pre-computed arrays of $\tilde{u}_{d_{c}}$ and $\tilde{w}$, $\tilde{u}_{d}(\tilde{x}, \tilde{r}, C_{t}, I_{0})$ can readily be calculated from the equation describing the Gaussian profile in the radial dimension.

## PyWake integration

### Vectorised calculations

The PyWake computational framework uses vectorised calculations along the following dimensions:
$$
  i: \textrm{source turbine;}\\
  j: \textrm{downwind turbine or other point;}\\
  k: \textrm{reference wind speed; and}\\
  l: \textrm{reference wind direction.}
$$

The EV wake model equations have been vectorised to compute matrices (arrays) of velocity deficit, wake width and added turbulence as a function of matrices of the required input data, including local free and waked wind speeds, local turbulence intensity, turbine downwind and crosswind spacings, rotor diameters and thrust coefficients.

### Rotor averaging and wake superposition

The integration of the EV wake model into the PyWake framework only required implementing the new single wake models. The models to account for variations across the rotor and to combine multiple wake effects could be adopted from existing implementation.

As a default, the EV deficit model has been implemented to use a grid rotor average model, thus estimating the effective wind speed of a waked turbine as an average of the predictions across the rotor.

The added turbulence model has been implemented under the assumption that the additional turbulence is constant across the wake. Using the wake width as an approximate measure of the limit to the region where the turbulence is elevated, an area overlap model has been used to weight the prediction by the proportion of a target turbine rotor that is within the limits of the wake from a source turbine.

The usual method for combining wake deficit estimates from multiple turbines with the EV model is to take the largest deficit rather than to let multiple deficits accumulate. The maximum is then taken is taken for each wind speed and wind direction bin. This approach has been adopted as the default implementation in the PyWake EV model. Accordingly, the wake wind speeds are scaled with the free wind speed values rather than with the effective (wake-affected) wind speeds.

The effective incident turbulence intensity is calculated as the root sum square of the ambient turbulence intensity and the estimated maximum wake added turbulence intensity (the estimate from the single turbine with the largest contribution). As noted in §\ref{wake_turbulence_model}, the estimated value is subsequently scaled to correspond to the reduced wind speed in the wake.

The approaches taken in the PyWake implementation are broadly in line with the WindFarmer and Openwind implementations, though not necessarily in all the detail.

## Comparison of results

As a validation of the solver and lookup table generator, a comparison of single wake centreline deficit results was made against the cases presented by Anderson (2011). Results calculated using the Openwind software were also included for reference, both using the standard EV model in the latest version and using the 'FastEddy' implementation of the simplified solution in the older Community Edition (CE) version. A plot showing the different results, which are all in good agreement, is presented below. In all of these results, the near wake mixing filter function is enabled. WindFarmer results are not included for this case, since WindFarmer always has the near wake mixing filter function disabled (i.e. it is not configurable).

<img src="images/ev_wake_recovery_results_comparison_mixing_function.png" alt="Comparison of single wake centreline deficit results when the near wake mixing filter function is enabled." style="width: 400px;"/>

A comparison of results for the model configuration without the near wake mixing filter function is presented in the following. Anderson (2011) did not present results for this case. The results of the PyWake EV model implementation are in close agreement with the Openwind results. The WindFarmer results differ significantly, likely due to having adopted a modified form of the original model equations. The Openwind Theory Manual noted significant differences in EV model results between Openwind and WindFarmer.

<img src="images/ev_wake_recovery_results_comparison_no_mixing_function.png" alt="Comparison of single wake centreline deficit results when the near wake mixing filter function is disabled." style="width: 400px;"/>

## References

Ainslie, J F. 1988. “Calculating the Flowfield in the Wake of Wind Turbines.” *J Wind Eng Ind Aerod* 27: 213–24.

Anderson, M. 2009. “Simplified Solution to the Eddy-Viscosity Wake Model.”

AWS Truewind. 2009. “Openwind Theoretical Basis and Validation.”

Garrad Hassan and Partners Ltd. 2014. “WindFarmer Theory Manual - April 2014.”

Hassan, U. 1992. “A Wind Tunnel Investigation of the Wake Structure Within Small Wind Turbine Farms.” Department of Energy.

Quarton, D C, and J F Ainslie. 1990. “Turbulence in Wind Turbine Wakes.” *Wind Eng* 14 (1): 15–23.

Taylor, G J. 1990. *Wake Measurements on the Nibe Wind Turbines in Denmark*. National Power, Technology; Environment Centre.

Vermeulen, P E J. 1980. “An Experimental Analysis of Wind Turbine Wakes.” Copenhagen.

Vermeulen, P E J, and P Builtjes. 1981. “Mathematical Modelling of Wake Interaction in Wind Turbine Arrays, Part 1.” TNO.

Vermeulen, P E J, and J Vijge. 1981. “Mathematical Modelling of Wake Interaction in Wind Turbine Arrays, Part 2.” TNO.